In [ ]:
from typing import Dict, Any

In [ ]:
from dotenv import load_dotenv
import os
from elasticsearch import helpers  # For bulk Data Uploading
from elasticsearch import Elasticsearch  # Base function for interacting with Elasticsearch

load_dotenv()
client = Elasticsearch("http://localhost:9200/", api_key=os.getenv('apikey'))

#test client
print(client.info())

### Creating the dictionnaries containing all the mappings

We read the mappings list from the metadata.csv file, which is generated externally from the shared excel file, and the list is then transformed into a dictionnary of mappings with the proper formatting.

In [6]:
import os
from pprint import pprint
from read_csv import *

# grab mappings list from the csv file, which should be in the same directory as the script
DIR_NAME = os.getcwd()
FILE_NAME = "metadata.csv"
FILE_PATH = os.path.join(DIR_NAME, FILE_NAME)
data, header = read_csv(FILE_PATH)

# create the dictionnary of mappings
mappings = create_mapping(data,header)
pprint(mappings)

['label', 'type', 'units', 'file']
{'Publication/reference': {'type': 'keyword'},
 'Tfloor': {'meta': {'file': '.in'}, 'type': 'float'},
 'alpha_rad': {'meta': {'file': '.in'}, 'type': 'float'},
 'code hash': {'meta': {'file': 'header.txt'}, 'type': 'keyword'},
 'dtmax': {'meta': {'file': '.in'}, 'type': 'float'},
 'eccentricity': {'meta': {'file': '.setup'}, 'type': 'float'},
 'equation of state': {'meta': {'file': '.in'}, 'type': 'integer'},
 'excitation_HI': {'meta': {'file': '.in'}, 'type': 'integer'},
 'f_acc': {'meta': {'file': '.in'}, 'type': 'float'},
 'icompanion_star': {'meta': {'file': '.setup'}, 'type': 'integer'},
 'icool_method': {'meta': {'file': '.in'}, 'type': 'integer'},
 'icooling': {'meta': {'file': '.in'}, 'type': 'integer'},
 'idust_opacity': {'meta': {'file': '.in'}, 'type': 'integer'},
 'iget_tdust': {'meta': {'file': '.in'}, 'type': 'integer'},
 'init wind velocity': {'meta': {'file': '.in', 'units': 'km/s'},
                        'type': 'float'},
 'isink_ra

Add other settings to the index parameters

In [ ]:
'''
"settings" is technically not needed if we are working on a simple local host, but can be changed to optimise search performance on a database that is hosted on a cluster and searched by multiple users.
"mappings" is required if you wish to explicitly map fields to specific values
'''

index_definition = {
    "settings": {  
        "number_of_shards": 1,
    },
    "mappings": {
        "properties": mappings
    }
}

Create the index using the parameters set above.

In [ ]:
''' For now we use a test index, but the name of the index should be changed later on'''

client.indices.create(index="test_index", body=index_definition)


# Kibana

Once the index is created in elasticsearch, in order to view it in Kibana, go to **Management** and on left menu bar, scroll down to the **Kibana** subsection, and click **Data Views**. From here, on the top right, click **Create data view** in order to integrate the new index into the Kibana interface. This will allow you to view how kibana interprets the index you have created.

### Load information from .setup and .in file




In [ ]:
def LoadSetupData(dir: str, prefix: str,index_definition) -> Dict[str, Any]:
    """ Load the prefix.in and prefix.setup files to get general information about the phantom model

    Args:
        dir (str): directory of the simulation
        prefix (str): prefix used for the files

    Returns:
        Dict[str, Any]: a dictionary containing the info from the setup files
        (!! check units, they are not all in SI or cgs)
    """

    # load the prefix.in & prefix.setup file
    setup = {}
    try:
        with open(os.path.join(dir,'%s.setup'%prefix), 'r') as f:
            lines = f.readlines()
            for string in lines:
                line = string.split()
                if len(line) != 0:
                    if line[0] != '#':
                        stringName = line[0]

                        # Names that have to be changed
                        # Floats
                        if stringName == 'primary_mass': stringName = 'primary mass'
                        elif stringName == 'primary_racc': stringName = 'primary Racc'
                        elif stringName == 'primary_Reff': stringName = 'primary Reff'
                        elif stringName == 'primary_Teff': stringName = 'primary Teff'
                        elif stringName == 'secondary_mass': stringName = 'secondary mass'
                        elif stringName == 'secondary_racc': stringName = 'secondary Racc'
                        elif stringName == 'secondary_Reff': stringName = 'secondary Reff'
                        elif stringName == 'secondary_Teff': stringName = 'secondary Teff'
                        elif stringName == 'semi_major_axis': stringName = 'semi-major axis'
                        elif stringName == 'wind_gamma' or stringName == "temp_exponent": stringName = 'wind polytropic index'
                        # # For triples
                        # elif stringName == 'secondary_mass': stringName = 'massComp_ini'
                        # elif stringName == 'binary2_a' : stringName = 'sma_in_ini'
                        # elif stringName == 'binary2_e' : stringName = 'ecc_in'
                        # elif stringName == 'secondary_racc': stringName = 'rAccrComp'
                        # elif stringName == 'accr2b' : stringName = 'rAccrComp_in'
                        # elif stringName == 'racc2b' : stringName = 'rAccrComp_in'
                        # if name is in our index dictionary, and it has type float, save it as a float
                        if (stringName in index_definition["mappings"]["properties"]) and (index_definition["mappings"]["properties"][stringName]['type']=='float'):
                            setup[stringName] = float(line[2])
                        elif (stringName in index_definition["mappings"]["properties"]) and (index_definition["mappings"]["properties"][stringName]['type']=='integer'):
                            setup[stringName] = int(line[2])
                        elif (stringName in index_definition["mappings"]["properties"]) and (index_definition["mappings"]["properties"][stringName]['type']=='string'):
                            setup[stringName] = str(line[2])

                        # # For triples
                        # if stringName == 'q2':
                        #     stringName = 'massComp_in_ini'
                        #     if stringName in index_definition:
                        #         setup[stringName] = float(line[2])*setup['primary mass']

                        # Boolean
                        if stringName == 'icompanion_star':
                            stringName = 'single_star'
                            if int(line[2]) == 0: setup[stringName] = True
                            else: setup[stringName] = False
                            stringName = 'triple_star'
                            if int(line[2]) == 2: setup[stringName] = True
                            else: setup[stringName] = False

    except FileNotFoundError:
        print('')
        print(" ERROR: No %s.setup file found!"%prefix)
        print('')
        exit()

    try:
        with open(os.path.join(dir,'%s.in'%prefix), 'r') as f:
            lines = f.readlines()
            for string in lines:
                line = string.split()
                if len(line) != 0:
                    if line[0] != '#':
                        stringName = line[0]

                        # Strings
                        # if stringName == 'logfile': setup[stringName] = str(line[2])
                        # elif stringName == 'dumpfile': setup[stringName] = str(line[2])
                        # elif stringName == 'twallmax': setup[stringName] = str(line[2])
                        # elif stringName == 'dtwallmax': setup[stringName] = str(line[2])

                        # Floats
                        # else:

                        if stringName == 'wind_mass_rate': stringName = 'wind mass rate'
                        elif stringName == 'wind_velocity': stringName = 'init wind velocity'
                        elif stringName == 'wind_inject_radius': stringName = 'wind inject radius'
                        elif stringName == 'wind_temperature': stringName = 'wind temperature'
                        elif stringName == 'mu': stringName = 'mean molecular weight'
                        elif stringName == 'ieos': stringName = 'equation of state'
                        elif stringName == 'outer_boundary': stringName = 'outer boundary'
                        # not sure what this is
                        if line[2]=='F': line[2] = 0
                        
                        if (stringName in index_definition["mappings"]["properties"]) and (index_definition["mappings"]["properties"][stringName]['type']=='float'):
                            setup[stringName] = float(line[2])
                        elif (stringName in index_definition["mappings"]["properties"]) and (index_definition["mappings"]["properties"][stringName]['type']=='integer'):
                            setup[stringName] = int(line[2])
                        elif (stringName in index_definition["mappings"]["properties"]) and (index_definition["mappings"]["properties"][stringName]['type']=='string'):
                            setup[stringName] = str(line[2])
    except FileNotFoundError:
        print('')
        print(" ERROR: No %s.in file found!"%prefix)
        print('')
        exit()

    return setup


In [ ]:
#To add 1 model
Dir = "/lhome/jolienm/Documents/TierModels/finalModelsHIcooling/v10e00"
prefix = "wind"
testData = LoadSetupData(Dir,prefix,index_definition)
client.index(index='test_index', body=testData)
del testData

In [ ]:
#To add multiple models
base_command = {
    "_index": 'test_index',
    "_op_type": "index"
}
operations = []
models = ['v10e00','v20e00','v05e00']
prefix = "wind"
for model in models:
    Dir = '/lhome/jolienm/Documents/TierModels/finalModelsHIcooling/'+str(model)
    modelData = LoadSetupData(Dir,prefix,index_definition)
    operations.append((base_command | {"_source": modelData}))
    del modelData
print(operations)
helpers.bulk(client, operations, refresh=True)
del operations

### Now look into Dashboards: Analytics - Dashboards to visualise data
Documentation: https://www.elastic.co/guide/en/kibana/current/create-a-dashboard-of-panels-with-web-server-data.html

